In [2]:
##Pseudo Code for Analysis
# Import pandas and pathlib
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path
%matplotlib inline
#import statsmodels.api as sm

In [3]:
#Read in APIs

In [4]:
#Read CSVs

In [5]:
#All Dates are by year

In [6]:
#Pct change by Zip
#Price Per Square foot by Zip
    #"Date" "Zip" "Price Per Square foot"
#Schools
    #"Date" "Zip" "School District" "Grade"
#Crime
    #"Date" "Zip" "Type" 
#Income Tax
    #"Date" "Zip" "Income Tax Paid" "Number of returns" Total Charitable Contributions" "Brackets"
#Real Estate Tax
    #"Date" "Zip" "Real Estate taxes paid" "personal property tax"
#Opportunity Zones
    #"Zip"
#Stray Animals
    #"Date" "Zip" "Number of Strays Reported"
#Population
    #"Date" "Zip" "Number of People"
#Construction Permits
    #"Date" "Zip" "Type (Residential vs Commercial)"
#Restaurant, Bars, & Clubs

In [7]:
# Construction Permits Correlation to pct_chg
def correlate_construction_permits(permits_df, pct_chg_df, groupby_month=True):
    # input: permits_df with date, zip, permit_count, permit_type (permit_type: Residential vs Commercial)
    # input: pct_chg_df with date, zip, pct_change
    # return dataframe with zip, correlation (correlation of number of permits to pct change by zip code)
    
    # set 'date' column type to date with year+month or with only year format (for easier groupby date, later)
    if groupby_month == True:
        
        # set 'date' column type to date with year+month only format
        permits_df['date'] = pd.to_datetime(permits_df['date'], format= '%Y%m')
        pct_chg_df['date'] = pd.to_datetime(pct_chg_df['date'], format= '%Y%m')
        
    elif groupby_month == False:
        
        # set 'date' column type to date with year only format
        permits_df['date'] = pd.to_datetime(permits_df['date'], format= '%Y', exact=False)
        pct_chg_df['date'] = pd.to_datetime(pct_chg_df['date'], format= '%Y', exact=False)

    
    #  for permits_df and pct_chg_df, groupby zip and date then calculate sum of each zip/date grouping
    permits_by_zipdate = permits_df.groupby(['zip','date']).sum()
    pct_chg_by_zipdate = pct_chg.groupby(['zip','date']).sum()
    
#     # TESTING ONLY: spot check data
#     print(permits_by_zipdate)
    
    #  combine permits_by_zipdate and pct_chg_by_zipdate by matching indexes
    combined_permits_pct_chg_df = pd.concat([permits_by_zipdate, pct_chg_by_zipdate], axis="columns", join="inner")

    # drop any null data rows
    combined_permits_pct_chg_df.dropna(inplace=True)
    
    
    # Set column names for use in initializing correlation_per_zip dataframe, for appending rows, and for resetting index
    cols = ['zip','permit_correlation']

    #initialize correlation_per_zip dataframe with cols
    correlation_per_zip_df = pd.DataFrame(columns=cols)

    # iterate through each zip
    for zip, df in combined_permits_pct_chg_df.groupby(level=0):

        # calculate correlation of this zip's permit count to pct_chg
        correlation_df = df.corr()

        # record specific correlation value from correlation dataframe
        correlation_value = correlation_df.loc['permit_count']['pct_chg']

        # create row/series based on this zip's correlation to append to  dataframe
        correlation_row = [{cols[0] : zip, cols[1] : correlation_value}]

        # append row to dataframe
        correlation_per_zip_df = correlation_per_zip_df.append(correlation_row)

    # reset  dataframe index to zip/cols[0]
    correlation_per_zip_df.set_index(cols[0], inplace=True)
 
    return correlation_per_zip_df  # return correlation_per_zip_df with columns: 'zip', 'permit_correlation'
    # return correlation_per_zip_df, permits_by_zipdate, pct_chg_by_zipdate

In [8]:
# Read in the pct_change data
pct_chg_data = Path("../resources/dan/sample_pct_chg_dan.csv")
pct_chg = pd.read_csv(pct_chg_data, sep='\t')

In [9]:
# Read in the construction permits data
permits_data = Path("../resources/dan/sample_permits_dan.csv")
permits = pd.read_csv(permits_data, sep='\t')

In [10]:
# Test permits to pct_chg correlation
permit_test_df = None
permit_test_df = correlate_construction_permits(permits, pct_chg, groupby_month=True)

In [11]:
# Print out test results for permits/pct_chg
permit_test_df.head()

,permit_correlation
zip,
77002,0.199488
77003,-0.091708
77004,0.025493


In [12]:
# Put the target (housing value -- MEDV) in another DataFrame
#ROI = pd.DataFrame(data.target, columns=["pct_change"])

# define the data/predictors as the pre-set feature names  
# ind_var_1 = pd.DataFrame(data.data, columns=data.feature_names)

In [13]:
## Without a constant

#X = ind_var_1["permit_count"]
#y = ROI["pct_change"]

# Note the difference in argument order
#model = sm.OLS(y, X).fit()
#predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
#model.summary()

In [15]:
combined_data = pd.concat([permits, pct_chg], axis="columns", join="inner").sort_index()
combined_data.dropna(inplace=True)
combined_data.head()

,date,zip,permit_count,permit_type,date,zip,pct_chg
0,2016-01-01,77002,327,Residential,2016-01-01,77002,0.07
1,2016-02-01,77002,136,Residential,2016-02-01,77002,0.09
2,2016-03-01,77002,289,Residential,2016-03-01,77002,-0.02
3,2016-04-01,77002,360,Residential,2016-04-01,77002,0.01
4,2016-05-01,77002,90,Residential,2016-05-01,77002,0.02
